In [1]:
import yfinance as yf
from pandas import DataFrame
import pandas as pd
from datetime import date
import openai
from typing import Dict, Any
import os
from tqdm import tqdm
from tr4der.data_loader import DataLoader
from tr4der.utils.gpt_helper import GptHelper
from tr4der.tr4der import Tr4der

inst = Tr4der("10 highest EBITDA companies for the past 5 years")





Loading data...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 10/10 [00:01<00:00,  7.87it/s]

None


In [3]:
inst.strategy_data